<a href="https://colab.research.google.com/github/elhamod/IS883/blob/main/Prompt_Engineering_Agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Recap of How to use OpenAI API for chat: [GitHub](https://github.com/elhamod/IS883/blob/main/Open_Api_Guide.ipynb)

In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 1.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
config_ini_location = '/content/drive/MyDrive/Colab Notebooks/IS883/OpenAI guide/config.ini' # Change this to point to the location of your config.ini file.

import configparser

config = configparser.ConfigParser()
config.read(config_ini_location)
openai_api_key = config['OpenAI']['API_KEY']

In [4]:
import openai

# Initialize the OpenAI API with your API key
openai.api_key = openai_api_key

# You can set up your API key by harcdcoding it here. It is a hacky and bad practice as others will see your secret key clearly and use your account. But, can be used for trying something quick and dirty
# openai.api_key = 'YOUR_OPENAI_API_KEY'

response = openai.Completion.create(
  engine="text-davinci-003",
  prompt="Translate the following English text to Spanish: 'Hello, how are you?'",
  max_tokens=50
)

print(response.choices[0].text.strip())

¡Hola, ¿cómo estás?


Now, let's use another package, [`langchain`](https://python.langchain.com/docs/get_started/introduction), that uses OpenAI API and allows for more advanced capabilities.

In [5]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.3 MB/s eta 0:00:00


In [6]:
from langchain.chat_models import ChatOpenAI

In [7]:
chat = ChatOpenAI(openai_api_key=openai_api_key) #temperature=0.0, model=llm_model

template_string = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template(template_string)

customer_style = """informal Spanish"""


customer_email = """
Hello, how are you?
"""


customer_messages = prompt_template.format_messages(
                    style=customer_style,
                    text=customer_email)

print(customer_messages)


customer_response = chat(customer_messages)


print(customer_response.content)

[HumanMessage(content='Translate the text that is delimited by triple backticks into a style that is informal Spanish. text: ```\nHello, how are you?\n```\n')]
Hola, ¿cómo estás?


Output extraction:

You can change your prompt to extract certain pieces of information

In [8]:
travel_template = """\
For the following text, extract the following information:

source: Where I am travelling from.

destination: Where I am travelling to.

airline: Which airline I am travelling with.

date: The date of travel in the format mm/dd/yyyy.

price: How much I paid for the ticket.

Format the output as JSON

text: {text}
"""

prompt_template = ChatPromptTemplate.from_template(travel_template)

In [9]:
essay = """\
On Veteran's day, I woke up reluctantly in my bed in Boston. I did not want to go to work.\
I decided to give myself time off and go on a vacation. I went on Expedia and bought a ticket to \
Houston to see my family. I took the plane, arrived safely, and spent a great logn weekend with my cousins. \
It was the best $500 I had spent in a while. The only downside is that JetBlue has lost my luggage. :(
"""

In [10]:
messages = prompt_template.format_messages(text=essay)
chat = ChatOpenAI(openai_api_key=openai_api_key)
response = chat(messages)
print(response)

content='{\n  "source": "Boston",\n  "destination": "Houston",\n  "airline": "JetBlue",\n  "date": "mm/dd/yyyy",\n  "price": "$500"\n}'


In [11]:
import json

json_object = json.loads(response.content)
json_object['source']

'Boston'

Now, let's learn how to use roles in OpenAI API

In [12]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = (
    "You are a teacher that is explaining advanced computer science concepts to someone in {degree}"
)
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Explain to me what {concept} is."
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)


chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

# get a chat completion from the formatted messages
print(chat(
    chat_prompt.format_prompt(
        degree="Computer science graduate program", concept="RNN"
    ).to_messages()
).content)

RNN stands for Recurrent Neural Network. It is a type of artificial neural network that is designed to process sequential data or data with a temporal component, such as time series data, speech, text, or music. 

Unlike feedforward neural networks, which process data in a one-way direction from input to output, RNNs have connections that allow information to flow in cycles, making them capable of capturing the temporal dependencies in the data. This cyclic nature enables RNNs to retain memory or context of previous inputs while processing the current input.

The basic building block of an RNN is a single neuron, also known as a memory cell or recurrent cell. This cell takes an input and produces an output, as well as feeds the output back to itself as an input for the next time step. This feedback loop allows the RNN to maintain an internal state or memory that can be influenced by previous inputs and impact future outputs.

One of the most common types of RNNs is the Long Short-Term 

Now, let's have a conversation AI

In [13]:
!pip install langchain

In [14]:
!pip install openai

In [15]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.prompts import MessagesPlaceholder


In [16]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferMemory()

template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    verbose=True,
    memory=memory,
)


In [17]:
conversation.prompt

PromptTemplate(input_variables=['history', 'input'], template='You are my buttler Alfred. You are talking to Batman.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [18]:
memory

ConversationBufferMemory()

In [19]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:

Human: What is my name?
AI:

> Finished chain.


'Your name is Batman.'

In [20]:
conversation.predict(input="How old am I?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI:

> Finished chain.


'You are 45 years old, sir.'

In [21]:
conversation.predict(input="No, my age is 39")



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI: You are 45 years old, sir.
Human: No, my age is 39
AI:

> Finished chain.


'Apologies for the mistake, sir. You are indeed 39 years old.'

In [22]:
print(memory.buffer)

Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI: You are 45 years old, sir.
Human: No, my age is 39
AI: Apologies for the mistake, sir. You are indeed 39 years old.


In [23]:
conversation.predict(input="When was I born?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI: You are 45 years old, sir.
Human: No, my age is 39
AI: Apologies for the mistake, sir. You are indeed 39 years old.
Human: When was I born?
AI:

> Finished chain.


'You were born in 1982, sir.'

In [24]:
conversation.predict(input="Who is my son?")



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI: You are 45 years old, sir.
Human: No, my age is 39
AI: Apologies for the mistake, sir. You are indeed 39 years old.
Human: When was I born?
AI: You were born in 1982, sir.
Human: Who is my son?
AI:

> Finished chain.


'Your son is Damian Wayne, sir.'

In [25]:
memory.buffer

'Human: What is my name?\nAI: Your name is Batman.\nHuman: How old am I?\nAI: You are 45 years old, sir.\nHuman: No, my age is 39\nAI: Apologies for the mistake, sir. You are indeed 39 years old.\nHuman: When was I born?\nAI: You were born in 1982, sir.\nHuman: Who is my son?\nAI: Your son is Damian Wayne, sir.'

In [26]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
    print(conversation.predict(input="Where are you now?"))
    print(cb)



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI: You are 45 years old, sir.
Human: No, my age is 39
AI: Apologies for the mistake, sir. You are indeed 39 years old.
Human: When was I born?
AI: You were born in 1982, sir.
Human: Who is my son?
AI: Your son is Damian Wayne, sir.
Human: Where are you now?
AI:

> Finished chain.
I am currently in the Batcave, sir.
Tokens Used: 139
	Prompt Tokens: 128
	Completion Tokens: 11
Successful Requests: 1
Total Cost (USD): $0.000214


In [27]:
with get_openai_callback() as cb:
    print(conversation.predict(input="Write a poem about the adventurs of Batman and Joker in 400 words"))
    print(cb)



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What is my name?
AI: Your name is Batman.
Human: How old am I?
AI: You are 45 years old, sir.
Human: No, my age is 39
AI: Apologies for the mistake, sir. You are indeed 39 years old.
Human: When was I born?
AI: You were born in 1982, sir.
Human: Who is my son?
AI: Your son is Damian Wayne, sir.
Human: Where are you now?
AI: I am currently in the Batcave, sir.
Human: Write a poem about the adventurs of Batman and Joker in 400 words
AI:

> Finished chain.
In the shadows of Gotham City's night,
A vigilante takes up the fight.
With cape and cowl, he roams the streets,
Protecting the innocent from evil's deceit.

Batman, the Dark Knight, strong and brave,
A hero determined to save.
His gadgets and skills, honed with care,
To bring justice to a world unfair.

Joker, the Clown Prince of Crime,
A twisted mind, beyond reason or rhyme.
His laugh

I want to limit the buffer to a window of 3 exchanges

In [28]:
from langchain.memory import ConversationBufferWindowMemory

In [29]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

memory = ConversationBufferWindowMemory(k=10)

template =  "You are my buttler Alfred. You are talking to Batman"
system_message = SystemMessagePromptTemplate.from_template(template=template)
PROMPT = PromptTemplate(input_variables=['history', 'input'], template=template + '.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

conversation = ConversationChain(
    llm=chat,
    prompt=PROMPT,
    verbose=True,
    memory=memory,
)

In [30]:
print(conversation.predict(input="What's my name?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:

Human: What's my name?
AI:

> Finished chain.
Your name is Batman, sir.


In [31]:
print(conversation.predict(input="Where am I?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Batman, sir.
Human: Where am I?
AI:

> Finished chain.
You are currently at Wayne Manor, sir.


In [32]:
print(conversation.predict(input="I am now in Metropolis."))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Batman, sir.
Human: Where am I?
AI: You are currently at Wayne Manor, sir.
Human: I am now in Metropolis.
AI:

> Finished chain.
My apologies, sir. You are currently in Metropolis.


In [33]:
print(conversation.predict(input="Can you send me Robin?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Batman, sir.
Human: Where am I?
AI: You are currently at Wayne Manor, sir.
Human: I am now in Metropolis.
AI: My apologies, sir. You are currently in Metropolis.
Human: Can you send me Robin?
AI:

> Finished chain.
I'm sorry, sir, but Robin is currently unavailable.


In [34]:
print(conversation.predict(input="Where am I?"))



> Entering new ConversationChain chain...
Prompt after formatting:
You are my buttler Alfred. You are talking to Batman.

Current conversation:
Human: What's my name?
AI: Your name is Batman, sir.
Human: Where am I?
AI: You are currently at Wayne Manor, sir.
Human: I am now in Metropolis.
AI: My apologies, sir. You are currently in Metropolis.
Human: Can you send me Robin?
AI: I'm sorry, sir, but Robin is currently unavailable.
Human: Where am I?
AI:

> Finished chain.
My apologies for the confusion, sir. You are currently in Metropolis.


Let's try a sequential chain

![picture](https://miro.medium.com/v2/resize:fit:828/format:webp/1*hdx24fJuQwWm1fT-ULGQhg.jpeg)

In [36]:
from langchain.chains import SimpleSequentialChain
from langchain.chains import LLMChain

In [37]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Give me the names of 10 different countries in {input}"
)

# Chain 1
chain_one = LLMChain(llm=chat, prompt=first_prompt)

In [38]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Order the names in this by country size: {input}"
)
# chain 2
chain_two = LLMChain(llm=chat, prompt=second_prompt)

In [39]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                            #  verbose=True
                                            )
continent = "North America"
overall_simple_chain.run(continent)

'1. United States\n2. Canada\n3. Mexico\n4. Cuba\n5. Haiti\n6. Dominican Republic\n7. Jamaica\n8. Guatemala\n9. Honduras\n10. Costa Rica'

Interesting to repeat multiple times and see how "incorrect" it can be

A somewhat more complicated chain:

In [40]:
from langchain.chains import SequentialChain

In [41]:
chat = ChatOpenAI(openai_api_key=openai_api_key)

In [42]:
problem = "\
John has some amount of apples, Sarah has double that amount, \
and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?\
"

In [43]:
# prompt template 1: translate to english
first_prompt = ChatPromptTemplate.from_template(
    "Convert the following problem into an equation in terms of x, where x is the number of apples John has:"
    "\n\n{problem}. Only give an equation"
)

# chain 1: input= Review and output= English_Review
chain_one = LLMChain(llm=chat, prompt=first_prompt,
                     output_key="Equation", verbose=True
                    )


In [44]:
# prompt template 1: translate to english
second_prompt = ChatPromptTemplate.from_template(
    "After solving the following equation in terms of x:"
    "\n\n{Equation}, only provide the response as `x=`"
)

# chain 1: input= Review and output= English_Review
chain_two = LLMChain(llm=chat, prompt=second_prompt,
                     output_key="Solution", verbose=True
                    )


In [45]:
# prompt template 1: translate to english
third_prompt = ChatPromptTemplate.from_template(
    "Now, narrate how you solved this {problem} to a 6 year old"
)

# chain 1: input= Review and output= English_Review
chain_three = LLMChain(llm=chat, prompt=third_prompt,
                     output_key="narration"
                    )

In [46]:

overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three],
    input_variables=["problem"],
    output_variables=["Equation", "Solution", "narration"],
    verbose=True
)

In [47]:
import langchain
langchain.debug = True

overall_chain(problem)

[chain/start] [1:chain:SequentialChain] Entering Chain run with input:
{
  "problem": "John has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?"
}
[chain/start] [1:chain:SequentialChain > 2:chain:LLMChain] Entering Chain run with input:
{
  "problem": "John has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?"
}
[llm/start] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: Convert the following problem into an equation in terms of x, where x is the number of apples John has:\n\nJohn has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?. Only give an equation"
  ]
}
[llm/end] [1:chain:SequentialChain > 2:chain:LLMChain > 3:llm:ChatOpenAI] [2.58s

{'problem': 'John has some amount of apples, Sarah has double that amount, and Mohannad has 3 apples. If they altogether have 12 apples, how man y does John have?',
 'Equation': 'Let x be the number of apples John has.\n\nSince Sarah has double the amount of apples John has, Sarah has 2x apples.\n\nTogether, John, Sarah, and Mohannad have 12 apples.\n\nSo, the equation can be written as:\n\nx + 2x + 3 = 12',
 'Solution': 'x = 12 - 3 - 2x\nx + 2x = 12 - 3\n3x = 9\nx = 9/3\nx = 3',
 'narration': "Well, let's solve this puzzle step by step!\n\nJohn has some apples, but we don't know how many yet. So, let's say John has X apples.\n\nSarah has double the amount of apples John has. So, if John has X apples, Sarah will have 2X apples.\n\nAnd Mohannad has 3 apples, which means Mohannad has 3 apples.\n\nNow, let's add up all the apples they have together. John has X apples, Sarah has 2X apples, and Mohannad has 3 apples. When we add them up, it equals 12 apples.\n\nSo, we can write it like this

Let's load a PDF and summarize the first page

In [48]:
!pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 4.9 MB/s eta 0:00:00


In [49]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/drive/MyDrive/MohannadCV.pdf")
pages = loader.load_and_split()

In [50]:
p = pages[0]

In [51]:
m = "Read the following page and summarize it in 50 words: "+ p.page_content

In [52]:
m

'Read the following page and summarize it in 50 words: Mohannad Elhamod\n/mobile_phone(206) 902 6177 •/envelopeelhamod@vt.edu •/globewordpress.cs.vt.edu/elhamod\n/linkedinmohannadelhamod\nEducation\nVirginia Tech, Blacksburg, VA May 2023\nPh.D. in Computer Science\nGPA: 3.83\nMcGill University, Montreal, QC April 2012\nM.Eng in Computer Engineering\nGPA: 3.83\nJordan University of Science and Technology, Irbid, Jordan June 2007\nB.Sc in Computer Engineering\nGPA: 84.4%. Rated Excellent\nResearch Interests\nInterested in Knowledge-Guided Machine Learning, and ML interpretability and visualization tools. Involved in\ninterdisciplinary projects that span a variety of scientiﬁc domains, including business, physics, and biology.\nResearch Experience\nDurability, Damage Tolerance and Reliability Branch, NASA June 2022 — August 2022\nResearch Intern\n/circle_blankTesting, developing and applying deep learning to demonstrate the use of physics-informed generative adversarial networks\nto learn

In [53]:
langchain.debug = False

chat = ChatOpenAI(openai_api_key=openai_api_key)
chat.predict(m)

# human_message_prompt = HumanMessagePromptTemplate.from_template(m)
# chat_prompt = ChatPromptTemplate.from_messages(
#     [human_message_prompt]
# )

# print(chat(
#     chat_prompt.format_prompt().to_messages()
# ).content)

'Mohannad Elhamod is a Ph.D. student in Computer Science at Virginia Tech. He has a strong research interest in Knowledge-Guided Machine Learning and ML interpretability and visualization tools. He has research experience in various domains, including aerospace materials and species modeling. He also has work experience as a software engineer at Microsoft, specifically in the Azure Monitoring and Microsoft Edge teams.'

Now, let's use agents

Math tool

In [57]:
!pip install -U wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=c9df68ea7c2a290e3775d7a0e936f49ca9fc7e8e9c244578120b8478229741ba
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [65]:
from langchain.agents import load_tools, initialize_agent

llm = ChatOpenAI(openai_api_key=openai_api_key) # temperature=0,  #model=llm_model
tools = load_tools(["wikipedia"], llm=llm) #,"llm-math"

agent= initialize_agent(
    tools,
    llm,
    # agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    # handle_parsing_errors=True,
    verbose = True)

# msg = "What is the 25% of 300?"
msg = "When was ChatGPT 4 released?"

agent(msg)



> Entering new AgentExecutor chain...
I should use Wikipedia to find the release date of ChatGPT 4.
Action: Wikipedia
Action Input: "ChatGPT 4 release date"
Observation: Page: ChatGPT
Summary: ChatGPT, which stands for Chat Generative Pre-trained Transformer, is a large language model-based chatbot developed by OpenAI and launched on November 30, 2022, which enables users to refine and steer a conversation towards a desired length, format, style, level of detail, and language. Successive prompts and replies, known as prompt engineering, are considered at each conversation stage as a context.ChatGPT is built upon either GPT-3.5 or GPT-4—members of OpenAI's proprietary series of generative pre-trained transformer (GPT) models, based on the transformer architecture developed by Google—and is fine-tuned for conversational applications using a combination of supervised and reinforcement learning techniques. ChatGPT was released as a freely available research preview, but due to its popula

{'input': 'When was ChatGPT 4 released?',
 'output': 'ChatGPT 4 was released on November 30, 2022.'}

In [66]:
from langchain.schema import HumanMessage

msgs = [HumanMessage(content=msg)]
llm(msgs).content

'As of September 2021, OpenAI has released ChatGPT-4-API, which is the latest version of the ChatGPT model. The specific release date of ChatGPT-4-API is not mentioned.'

Define your own tool

The descriprion is what really tells the agent which tool to use.

In [ ]:
#!pip install DateTime

In [67]:
from langchain.agents import tool
from datetime import date

@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [78]:
agent= initialize_agent(
    [time], #tools +
    llm,
    # agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose = True)

In [79]:
agent("I ate a date. Will I be OK?")



> Entering new AgentExecutor chain...
The question is asking about the person's well-being after eating a date. However, this question cannot be answered with the tools given.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I cannot answer this question with the given tools.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I cannot answer this question with the given tools.
Final Answer: The question cannot be answered.

> Finished chain.


{'input': 'I ate a date. Will I be OK?',
 'output': 'The question cannot be answered.'}